<p align="left">
  <img src="https://raw.githubusercontent.com/python35/IINTS-SDK/main/img/iints_logo.png" width="160">
</p>

# Baseline Comparison & Metrics
[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/python35/IINTS-SDK/blob/main/examples/notebooks/04_Baseline_and_Metrics.ipynb)

**Goal:** compare your algorithm against a baseline controller and compute clinical metrics.

**You will learn:**
- Run a baseline comparison
- Calculate TIR, GMI, CV, LBGI, HBGI


In [1]:
from __future__ import annotations
from pathlib import Path
import os
import sys
import subprocess


def _find_repo_root() -> Path | None:
    for root in [Path.cwd(), *Path.cwd().parents]:
        if (root / "pyproject.toml").exists() and (root / "src").exists():
            return root
    return None

repo_root = _find_repo_root()
if repo_root is None:
    try:
        import google.colab  # type: ignore
        in_colab = True
    except Exception:
        in_colab = False

    if not in_colab:
        raise RuntimeError("Run this notebook inside the IINTS-SDK repo or on Colab.")

    if not Path("IINTS-SDK").exists():
        subprocess.check_call(["git", "clone", "https://github.com/python35/IINTS-SDK.git"])
    repo_root = Path("IINTS-SDK").resolve()

os.chdir(repo_root)
sys.path.insert(0, str(repo_root / "src"))
print("Repo root:", repo_root)


Repo root: /home/runner/work/IINTS-SDK/IINTS-SDK


In [2]:
import iints
from iints.presets import get_preset
from iints.core.algorithms.fixed_basal_bolus import FixedBasalBolus
from iints.analysis.clinical_metrics import ClinicalMetricsCalculator

preset = get_preset("baseline_t1d")
scenario = dict(preset["scenario"])
scenario["stress_events"] = [e for e in scenario.get("stress_events", []) if e.get("event_type") != "exercise"]

algorithm = FixedBasalBolus(settings={"fixed_basal_rate": 0.4, "carb_ratio": 12.0})


In [3]:
from iints.validation import load_patient_config_by_name

patient_config = load_patient_config_by_name(preset["patient_config"]).model_dump()
patient_config.update(
    {
        "glucose_decay_rate": 0.01,
        "basal_insulin_rate": 0.4,
        "insulin_sensitivity": 60.0,
        "initial_glucose": 150.0,
    }
)


In [4]:
outputs = iints.run_simulation(
    algorithm=algorithm,
    scenario=scenario,
    patient_config=patient_config,
    duration_minutes=360,
    time_step=preset["time_step_minutes"],
    seed=11,
    output_dir=None,
    compare_baselines=True,
)

outputs["baseline_comparison"]


Simulation terminated early: Critical failure: glucose < 40.0 mg/dL for 30 minutes.


Simulation terminated early: Critical failure: glucose < 40.0 mg/dL for 30 minutes.


Simulation terminated early: Critical failure: glucose < 40.0 mg/dL for 30 minutes.


{'reference': 'Standard PID',
 'rows': [{'algorithm': 'FixedBasalBolus',
   'tir_70_180': np.float64(57.692307692307686),
   'tir_below_70': np.float64(42.30769230769231),
   'tir_above_180': np.float64(0.0),
   'bolus_interventions': 12,
   'total_violations': 12},
  {'algorithm': 'Standard PID',
   'tir_70_180': np.float64(55.00000000000001),
   'tir_below_70': np.float64(45.0),
   'tir_above_180': np.float64(0.0),
   'bolus_interventions': 5,
   'total_violations': 14},
  {'algorithm': 'Standard Pump',
   'tir_70_180': np.float64(54.166666666666664),
   'tir_below_70': np.float64(45.83333333333333),
   'tir_above_180': np.float64(0.0),
   'bolus_interventions': 12,
   'total_violations': 12}]}

## Metrics for the primary run


In [5]:
calc = ClinicalMetricsCalculator()
metrics = calc.calculate(
    glucose=outputs["results"]["glucose_actual_mgdl"],
    duration_hours=outputs["results"]["time_minutes"].max() / 60.0,
)
metrics.to_dict()


{'tir_70_180': np.float64(57.692307692307686),
 'tir_70_140': np.float64(50.0),
 'tir_70_110': np.float64(30.76923076923077),
 'tir_below_70': np.float64(42.30769230769231),
 'tir_below_54': np.float64(30.76923076923077),
 'tir_above_180': np.float64(0.0),
 'tir_above_250': np.float64(0.0),
 'cv': np.float64(51.13678977442155),
 'sd': 40.03453849003339,
 'gmi': np.float64(5.182675552974583),
 'mean_glucose': 78.28911174642904,
 'median_glucose': 77.27372876333236,
 'hi': np.float64(18.3476708890081),
 'lbgi': np.float64(2.1607064799143525),
 'hbgi': np.float64(0.026549239319969404),
 'readings_per_day': np.float64(299.52),
 'data_coverage': 100}

### Recap
You now have baseline comparison plus clinical metrics in one place.
